In [8]:
#Library:

import sys
import numpy as np
import math
import random
import gzip
import os
import re

#set the cosmology
from astropy.cosmology import FlatLambdaCDM
from astropy import constants as const
import astropy.units as u
import matplotlib.pyplot as plt

In [4]:
Hubble_h = 0.673 #hubble constant
Omega_M = 0.315 #barionic enegy density
Omega_Lambda = 0.683 #Dark energy
cosmo = FlatLambdaCDM(H0=Hubble_h*100, Om0=Omega_M)

#universal constant:
c=const.c.to('Mpc/ s') /(u.Mpc)*(u.s) # Mpc/s
G=const.G.to('Mpc3 /M_sun s2') /(u.Mpc**3)*(u.s**2)*(u.M_sun)

def sort_file_lines(input_file, output_file, sort_order):
    # Read the contents of the input file into a list
    with open(input_file, 'r') as file:
        lines = file.readlines()
    
    # Remove any leading/trailing whitespace and convert lines to float values
    lines = [list(map(float, line.strip().split())) for line in lines]
    
    # Sort the lines based on the provided sort_order
    sorted_lines = [lines[i] for i in sort_order]

    #keep only the first 10 columns
    sorted_lines = [line[:10] for line in sorted_lines]

    #add two columns at the end intial phase and pericenter angle columns with random values between 0 and 2pi
    sorted_lines = [line + [np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi)] for line in sorted_lines]
    
    # Write the sorted lines to the output file
    with open(output_file, 'w') as file:
        for line in sorted_lines:
            file.write(' '.join(str(value) for value in line))
            file.write('\n')
            

#Observation time
T=60.*60.*24.*365.*30.
#freq bin:
DF=1/T
#freq yr-1
Fyr=1/(60*60*24*365)

#Mchirp
def Mc_f(M1,M2):
    return (M1*M2)**(3./5.) / ((M1+M2)**(1./5.))

#characteristic amplitude:
def A_f(Mc,f,d,z):     
    return 2.*(G*Mc)**(5/3)*(2*np.pi*f*(1.+z))**(2./3.) / (pow(c,4)*d)

#inclination angle function:
def a_f(i):
    return 1. + ((np.cos(i))**2.)

def b_f(i):
    return -2.*np.cos(i)

def meanAng_f(i):
    return np.sqrt( (1./2.) *( ( a_f(i)*a_f(i) + b_f(i)*b_f(i) ) ))
    
model_name = "MBHBs_David_103_Universe_105_Realization_101.dat"

In [5]:
# Read in the file
with open(model_name, 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('inf', '0.0')

# Write the file out again
with open('fix_' +  model_name, 'w') as file:
  file.write(filedata)

In [6]:
#read the L-galaxies output
fname = 'fix_' + model_name
fp=open(fname,'r')
Nbinary= len(fp.readlines()) #binaries number
#fiducial model read the data 
M1 =       np.genfromtxt(fname,usecols=0,max_rows=Nbinary)  #Mass first SMBH             0
M2 =       np.genfromtxt(fname,usecols=1,max_rows=Nbinary)  #Mass seconf SMBH            1
f_gw =     np.genfromtxt(fname,usecols=2,max_rows=Nbinary)  #frequency of the binary     2
a =        np.genfromtxt(fname,usecols=3,max_rows=Nbinary)  #binary separation           3
e =        np.genfromtxt(fname,usecols=4,max_rows=Nbinary)  #eccentricity of the binary  4
z =        np.genfromtxt(fname,usecols=5,max_rows=Nbinary)  #redshift of the binary      5
i =        np.genfromtxt(fname,usecols=6,max_rows=Nbinary)  #inclination angle           6
PSI0 =     np.genfromtxt(fname,usecols=7,max_rows=Nbinary)  # polarizaion angle          7
RA =       np.genfromtxt(fname,usecols=8,max_rows=Nbinary)  #right ascension             9
DEC =      np.genfromtxt(fname,usecols=9,max_rows=Nbinary)  #Declination                 10
fp.close()

#Chirp mass
Mc=Mc_f(M1,M2)
#comoving distance:
D = cosmo.comoving_distance(z).cgs.to('Mpc')/(u.Mpc)

#obs obital freq self const:
f = (1/(2*math.pi)) * np.sqrt(  G*(M1+M2)/(a*a*a)  ) / (1+z)
f = f.value

#GW amplitude
A = A_f(Mc,f,D,z)
hc_2 = A*A*meanAng_f(i)*meanAng_f(i)*f*T
hc=np.sqrt(hc_2)

#sorted index:
ix = np.argsort(hc)[::-1]

#sorted catalog name:
sort_cat_name='SORT_'+ model_name

#Sort catalogue
in_file = 'fix_' + model_name
#path to folder containing the sorted catalogs

out_file = sort_cat_name
sort_order = ix  # Replace with your desired sort order

sort_file_lines(in_file, out_file, sort_order)


os.remove('fix_'+ model_name)

431914
